# 🔬 Analyse Comparative DeepSeek vs NSM-Greimas

**Projet** : Panini Research - Semantic Primitives  
**Date** : 12 novembre 2025  
**Objectif** : Valider convergence entre modèle symbolique (NSM-Greimas) et neural (DeepSeek)

---

## 📋 Hypothèses Testées

| Hypothèse | Métrique | Seuil | Statut |
|-----------|----------|-------|--------|
| **H1** - Clustering catégoriel | Pureté | > 0.7 | 🔄 Test |
| **H2** - Carrés sémiotiques | Validation | > 70% | 🔄 Test |
| **H3** - Isotopies convergentes | Corrélation | > 0.6 | 🔄 Test |
| **H4** - Reconstruction linéaire | R² | > 0.5 | 🔄 Test |

---

## ⚡ Avantages Colab Pro

- **GPU** : NVIDIA A100/V100 (encodage 100x plus rapide)
- **RAM** : 52 GB (corpus 1000+ phrases)
- **Runtime** : 24h continues
- **Stockage** : Google Drive intégré

## 🔧 Setup Environnement

In [ ]:
# Vérifier GPU disponible
import torch

if torch.cuda.is_available():
    print(f"✅ GPU disponible : {torch.cuda.get_device_name(0)}")
    print(f"   VRAM : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ Pas de GPU détecté - Activer GPU dans Runtime > Change runtime type")

In [ ]:
# Installer dépendances
!pip install -q openai matplotlib seaborn scikit-learn scipy plotly numpy pandas

print("✅ Dépendances installées")

In [ ]:
# Monter Google Drive pour sauvegardes
from google.colab import drive
import os

drive.mount('/content/drive')

# Créer dossier résultats
output_dir = '/content/drive/MyDrive/Panini/DeepSeek_Analysis'
os.makedirs(output_dir, exist_ok=True)

print(f"✅ Google Drive monté")
print(f"📁 Résultats : {output_dir}")

In [ ]:
# Cloner repo Panini Research
!git clone https://github.com/stephanedenis/Panini-Research.git
%cd Panini-Research/semantic-primitives

print("✅ Repo cloné")
!ls -la

## 🔑 Configuration API DeepSeek

**IMPORTANT** : Stocker votre clé API dans les secrets Colab :

1. Cliquer sur 🔑 (icône clé) dans la barre latérale gauche
2. Ajouter secret : `DEEPSEEK_API_KEY` = votre clé
3. Activer "Notebook access"

**Obtenir clé API** : https://platform.deepseek.com/api_keys

In [ ]:
# Charger clé API depuis secrets Colab
from google.colab import userdata

try:
    DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
    print("✅ Clé API DeepSeek chargée")
    print(f"   Préfixe : {DEEPSEEK_API_KEY[:8]}...")
except Exception as e:
    print("❌ Erreur : Clé API non trouvée dans secrets Colab")
    print("   → Ajouter DEEPSEEK_API_KEY dans les secrets (🔑)")
    DEEPSEEK_API_KEY = None

## 📦 Import Modules Panini

In [ ]:
# Ajouter chemin modules
import sys
sys.path.append('/content/Panini-Research/semantic-primitives/panlang')
sys.path.append('/content/Panini-Research/semantic-primitives/analysis-scripts')

# Imports Panini
from nsm_primitives_complet import NSM_PRIMITIVES
from greimas_nsm_extension import ReconstructeurGreimasNSM
from deepseek_analyzer import (
    ClientDeepSeek, 
    ConfigDeepSeek, 
    ModeleDeepSeek,
    AnalyseurConvergence
)

# Imports standard
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
from datetime import datetime

print("✅ Modules importés")
print(f"   - NSM primitives : {len(NSM_PRIMITIVES)}")
print(f"   - Reconstructeur Greimas chargé")
print(f"   - Analyseur convergence prêt")

## 🚀 Initialisation Analyseur

In [ ]:
# Configuration DeepSeek
config = ConfigDeepSeek(
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com",
    model=ModeleDeepSeek.CHAT,
    temperature=0.0  # Déterministe pour reproductibilité
)

# Client DeepSeek (mode réel si clé disponible)
client = ClientDeepSeek(config)

# Analyseur convergence
analyseur = AnalyseurConvergence(client)

print("✅ Analyseur initialisé")
print(f"   Mode : {'RÉEL (API DeepSeek)' if DEEPSEEK_API_KEY else 'SIMULATION'}")

---

# 🔬 EXPÉRIENCE 1 : Clustering Primitives NSM

**Objectif** : Vérifier si les 12 catégories NSM sont linéairement séparables dans l'espace DeepSeek

**Protocole** :
1. Encoder 60 primitives NSM avec DeepSeek
2. Réduction dimensionnelle t-SNE (4096 → 2D)
3. Clustering K-means (k=12 catégories)
4. Mesure pureté + silhouette

**Seuil validation** : Pureté > 0.7, Silhouette > 0.5

In [ ]:
# Encoder primitives NSM
print("🔄 Encodage 60 primitives NSM avec DeepSeek...")
print("   (Durée estimée : 2-3 min avec GPU)\n")

embeddings_primitives = analyseur.encoder_primitives_nsm()

print(f"\n✅ {len(embeddings_primitives)} primitives encodées")
print(f"   Dimension embeddings : {embeddings_primitives[0]['embedding'].shape}")

In [ ]:
# Visualisation t-SNE
print("🔄 Calcul t-SNE et visualisation...\n")

coords_2d, categories = analyseur.visualiser_tsne(
    embeddings_primitives,
    output_path=f"{output_dir}/tsne_primitives_nsm_real.png"
)

plt.show()
print(f"\n✅ Visualisation sauvegardée : {output_dir}/tsne_primitives_nsm_real.png")

In [ ]:
# Évaluation clustering
print("🔄 Évaluation clustering K-means...\n")

metriques = analyseur.evaluer_clustering(embeddings_primitives)

print(f"\n📊 RÉSULTATS EXPÉRIENCE 1")
print("=" * 50)
print(f"Pureté clustering    : {metriques['purete']:.3f} (seuil > 0.7)")
print(f"Silhouette score     : {metriques['silhouette']:.3f} (seuil > 0.5)")
print(f"Nombre clusters      : {metriques['n_clusters']}")

# Validation
h1_validee = metriques['purete'] > 0.7 and metriques['silhouette'] > 0.5
print(f"\n{'✅' if h1_validee else '❌'} H1 - Clustering catégoriel : {'VALIDÉE' if h1_validee else 'RÉFUTÉE'}")

---

# 🔬 EXPÉRIENCE 2 : Carrés Sémiotiques Greimas

**Objectif** : Vérifier si les oppositions Greimas respectent un ordre géométrique dans DeepSeek

**Protocole** :
1. Encoder 20 paires de contraires (BON/MAUVAIS, etc.)
2. Calculer 4 positions du carré : S1, S2, non-S1, non-S2
3. Mesurer distances cosinus
4. Valider ordre : d(contraire) > d(contradiction) > d(subcontraire)

**Seuil validation** : > 70% des carrés valident structure

In [ ]:
# Analyser carrés sémiotiques
print("🔄 Analyse 20 carrés sémiotiques...")
print("   (Durée estimée : 3-4 min avec GPU)\n")

analyse_carres = analyseur.analyser_carres_semiotiques()

print(f"\n📊 RÉSULTATS EXPÉRIENCE 2")
print("=" * 50)
print(f"Carrés analysés      : {len(analyse_carres['resultats'])}")
print(f"Carrés valides       : {sum(1 for r in analyse_carres['resultats'] if r['structure_valide'])}")
print(f"Taux validation      : {analyse_carres['taux_validation']:.1%}")

# Validation
h2_validee = analyse_carres['taux_validation'] > 0.7
print(f"\n{'✅' if h2_validee else '❌'} H2 - Carrés sémiotiques : {'VALIDÉE' if h2_validee else 'RÉFUTÉE'}")

In [ ]:
# Heatmap carrés
print("\n🔄 Génération heatmap carrés sémiotiques...\n")

analyseur.visualiser_heatmap_carres(
    analyse_carres,
    output_path=f"{output_dir}/heatmap_carres_real.png"
)

plt.show()
print(f"✅ Heatmap sauvegardée : {output_dir}/heatmap_carres_real.png")

---

# 🔬 EXPÉRIENCE 3 : Isotopies Corpus Littéraire

**Objectif** : Comparer isotopies NSM avec features principales DeepSeek

**Protocole** :
1. Corpus littéraire (Camus, Hugo, Proust, Saint-Exupéry)
2. Détection isotopies NSM (fréquences primitives)
3. Encodage DeepSeek + PCA
4. Corrélation isotopies NSM ↔ features PCA

**Seuil validation** : Corrélation > 0.6

In [ ]:
# Charger corpus littéraire
sys.path.append('/content/Panini-Research/semantic-primitives/tests')
from test_corpus_litteraire import CORPUS_CAMUS, CORPUS_HUGO, CORPUS_PROUST, CORPUS_EXUPERY

# Corpus complet (105 phrases)
corpus_complet = CORPUS_CAMUS + CORPUS_HUGO + CORPUS_PROUST + CORPUS_EXUPERY

print(f"📚 Corpus littéraire chargé")
print(f"   - Camus : {len(CORPUS_CAMUS)} phrases")
print(f"   - Hugo : {len(CORPUS_HUGO)} phrases")
print(f"   - Proust : {len(CORPUS_PROUST)} phrases")
print(f"   - Saint-Exupéry : {len(CORPUS_EXUPERY)} phrases")
print(f"   - TOTAL : {len(corpus_complet)} phrases")

In [ ]:
# Analyser isotopies (avec GPU)
print("\n🔄 Analyse isotopies corpus complet...")
print("   (Durée estimée : 5-7 min avec GPU)\n")

analyse_isotopies = analyseur.analyser_isotopies_corpus(
    corpus_complet,
    nom_corpus="Corpus Littéraire Complet (105 phrases)"
)

print(f"\n📊 RÉSULTATS EXPÉRIENCE 3")
print("=" * 50)
print(f"Phrases analysées    : {len(corpus_complet)}")
print(f"Isotopies détectées  : {len(analyse_isotopies['isotopies_nsm'])}")
print(f"Variance PCA         : {analyse_isotopies['variance_expliquee'].sum():.1%}")

# Top isotopies avec corrélations
print(f"\n🔍 Top 10 isotopies NSM :")
top_isotopies = sorted(
    analyse_isotopies['isotopies_nsm'].items(),
    key=lambda x: x[1],
    reverse=True
)[:10]

for prim, freq in top_isotopies:
    if prim in analyse_isotopies['correlations']:
        corr = analyse_isotopies['correlations'][prim]['max_correlation']
        print(f"  {prim:15s} : freq={freq:3d}, corr={corr:.3f}")

# Validation
correlations_fortes = [c['max_correlation'] for c in analyse_isotopies['correlations'].values() if c['max_correlation'] > 0.6]
h3_validee = len(correlations_fortes) >= len(analyse_isotopies['correlations']) * 0.5

print(f"\n{'✅' if h3_validee else '❌'} H3 - Isotopies convergentes : {'VALIDÉE' if h3_validee else 'RÉFUTÉE'}")
print(f"   Isotopies r > 0.6 : {len(correlations_fortes)}/{len(analyse_isotopies['correlations'])}")

---

# 📊 Visualisation 3D Interactive (Plotly)

Projection t-SNE 3D des primitives NSM dans l'espace DeepSeek

In [ ]:
import plotly.graph_objects as go
from sklearn.manifold import TSNE

# t-SNE 3D
print("🔄 Calcul t-SNE 3D (peut prendre 2-3 min)...\n")

embeddings_array = np.array([item['embedding'] for item in embeddings_primitives])
labels = [item['primitive'] for item in embeddings_primitives]
categories = [item['categorie'] for item in embeddings_primitives]

tsne_3d = TSNE(n_components=3, perplexity=30, max_iter=1000, random_state=42)
coords_3d = tsne_3d.fit_transform(embeddings_array)

# Couleurs par catégorie
categories_uniques = list(set(categories))
color_map = {cat: i for i, cat in enumerate(categories_uniques)}
colors = [color_map[cat] for cat in categories]

# Graphique 3D interactif
fig = go.Figure(data=[go.Scatter3d(
    x=coords_3d[:, 0],
    y=coords_3d[:, 1],
    z=coords_3d[:, 2],
    mode='markers+text',
    text=labels,
    marker=dict(
        size=8,
        color=colors,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Catégorie", tickvals=list(range(len(categories_uniques))), ticktext=categories_uniques)
    ),
    textposition="top center",
    textfont=dict(size=8)
)])

fig.update_layout(
    title="Primitives NSM dans l'espace DeepSeek (3D interactif)",
    scene=dict(
        xaxis_title='Dimension 1',
        yaxis_title='Dimension 2',
        zaxis_title='Dimension 3'
    ),
    width=1000,
    height=800
)

fig.show()

# Sauvegarder HTML interactif
html_path = f"{output_dir}/viz_3d_interactive.html"
fig.write_html(html_path)
print(f"\n✅ Visualisation 3D sauvegardée : {html_path}")
print("   → Ouvrir ce fichier dans navigateur pour interaction complète")

---

# 💾 Sauvegarde Résultats

In [ ]:
# Préparer résultats pour sauvegarde
resultats_complets = {
    'metadata': {
        'date': datetime.now().isoformat(),
        'mode': 'RÉEL' if DEEPSEEK_API_KEY else 'SIMULATION',
        'modele': 'deepseek-chat',
        'corpus_size': len(corpus_complet)
    },
    'experience_1': {
        'purete': float(metriques['purete']),
        'silhouette': float(metriques['silhouette']),
        'n_clusters': int(metriques['n_clusters']),
        'hypothese_validee': h1_validee
    },
    'experience_2': {
        'taux_validation': float(analyse_carres['taux_validation']),
        'carres_valides': sum(1 for r in analyse_carres['resultats'] if r['structure_valide']),
        'carres_total': len(analyse_carres['resultats']),
        'hypothese_validee': h2_validee
    },
    'experience_3': {
        'isotopies_detectees': len(analyse_isotopies['isotopies_nsm']),
        'variance_pca': float(analyse_isotopies['variance_expliquee'].sum()),
        'correlations_fortes': len(correlations_fortes),
        'hypothese_validee': h3_validee,
        'top_isotopies': dict(top_isotopies)
    }
}

# Sauvegarder JSON
json_path = f"{output_dir}/resultats_deepseek_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(resultats_complets, f, indent=2, ensure_ascii=False)

print(f"✅ Résultats sauvegardés : {json_path}")

# Sauvegarder embeddings (pour analyses futures)
embeddings_path = f"{output_dir}/embeddings_primitives.npy"
np.save(embeddings_path, embeddings_array)
print(f"✅ Embeddings sauvegardés : {embeddings_path}")

---

# 📊 RAPPORT FINAL

In [ ]:
print("\n" + "=" * 70)
print("🔬 ANALYSE COMPARATIVE DEEPSEEK vs NSM-GREIMAS")
print("=" * 70)

print(f"\n📅 Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🖥️ Mode : {'RÉEL (API DeepSeek)' if DEEPSEEK_API_KEY else 'SIMULATION'}")
print(f"📚 Corpus : {len(corpus_complet)} phrases littéraires")

print("\n" + "=" * 70)
print("RÉSULTATS EXPÉRIENCES")
print("=" * 70)

print(f"\n{'✅' if h1_validee else '❌'} EXPÉRIENCE 1 : Clustering Primitives NSM")
print(f"   Pureté          : {metriques['purete']:.3f} (seuil > 0.7)")
print(f"   Silhouette      : {metriques['silhouette']:.3f} (seuil > 0.5)")
print(f"   → H1 : {'VALIDÉE' if h1_validee else 'RÉFUTÉE'}")

print(f"\n{'✅' if h2_validee else '❌'} EXPÉRIENCE 2 : Carrés Sémiotiques Greimas")
print(f"   Taux validation : {analyse_carres['taux_validation']:.1%} (seuil > 70%)")
print(f"   Carrés valides  : {sum(1 for r in analyse_carres['resultats'] if r['structure_valide'])}/{len(analyse_carres['resultats'])}")
print(f"   → H2 : {'VALIDÉE' if h2_validee else 'RÉFUTÉE'}")

print(f"\n{'✅' if h3_validee else '❌'} EXPÉRIENCE 3 : Isotopies Corpus Littéraire")
print(f"   Isotopies r>0.6 : {len(correlations_fortes)}/{len(analyse_isotopies['correlations'])}")
print(f"   Variance PCA    : {analyse_isotopies['variance_expliquee'].sum():.1%}")
print(f"   → H3 : {'VALIDÉE' if h3_validee else 'RÉFUTÉE'}")

print("\n" + "=" * 70)
print("CONCLUSION")
print("=" * 70)

hypotheses_validees = sum([h1_validee, h2_validee, h3_validee])
print(f"\nHypothèses validées : {hypotheses_validees}/3")

if hypotheses_validees >= 2:
    print("\n✅ CONVERGENCE VALIDÉE")
    print("   Les modèles NSM-Greimas et DeepSeek convergent significativement.")
elif hypotheses_validees == 1:
    print("\n⚠️ CONVERGENCE PARTIELLE")
    print("   Les modèles convergent partiellement, divergences importantes.")
else:
    print("\n❌ DIVERGENCE")
    print("   Les modèles capturent des réalités sémantiques différentes.")

print("\n" + "=" * 70)
print("FICHIERS GÉNÉRÉS")
print("=" * 70)
print(f"\n📁 Dossier : {output_dir}")
print("\nVisualisations :")
print("  - tsne_primitives_nsm_real.png")
print("  - heatmap_carres_real.png")
print("  - viz_3d_interactive.html (ouvrir dans navigateur)")
print("\nDonnées :")
print("  - resultats_deepseek_YYYYMMDD_HHMMSS.json")
print("  - embeddings_primitives.npy")

print("\n" + "=" * 70)
print("✅ ANALYSE TERMINÉE")
print("=" * 70)

---

# 🚀 Prochaines Étapes

## Actions Recommandées

1. **Télécharger résultats** depuis Google Drive
2. **Mettre à jour rapport** avec résultats API réelle
3. **Analyser divergences** (si hypothèses réfutées)
4. **Expérience 4** : Reconstruction linéaire (régression DeepSeek → NSM)
5. **Publication** : Rédiger article ACL 2026

## Corpus Étendu (Optionnel)

Pour analyse plus robuste, charger corpus large :

```python
# Télécharger depuis Google Drive
corpus_1000 = pd.read_csv('/content/drive/MyDrive/Panini/corpus_1000_phrases.csv')

# Relancer Expérience 3
analyse_large = analyseur.analyser_isotopies_corpus(
    corpus_1000['phrase'].tolist(),
    nom_corpus="Corpus Large (1000+ phrases)"
)
```

## Comparaison Multi-Modèles

Tester convergence avec autres LLMs :

- OpenAI GPT-4
- Anthropic Claude
- Google Gemini
- Meta Llama 3

**Question** : Quel modèle converge le plus vers NSM-Greimas ?